In [1]:
import gurobipy   as     gp
from   gurobipy   import GRB
from drawpointFunction  import drawpointFunction
from globalFunctions import getNumberOfBlocksInADimension
import pandas as pd

In [28]:
class UndergroundModel:
    #Underground Model + Crown Pillar Restrictions.
    def __init__(self, database, numberOfPeriods):
        self.database = database
        self.numberOfPeriods = numberOfPeriods
        self.DP_init = 0       #### Tipo de extracción
        self.desc = 0.1
        self.colHeight = 300
        self.pos_x = 430     
        self.pos_y = 550     
        self.pos_z = 780
        self.pos_x_f = 730     
        self.pos_y_f = 910     
        self.p_t = 3791.912
        self.epsilon = 1
        self.orientationToExtractTheDrawpoints = 0

    def execute(self):
        self.getUndergroundVariablesFromCSV()
        self.getUndergroundInfo()
        self.setUndergroundParameters()
        self.setUndergroundMineLimits()
        self.setUndergroundVariables()
        self.setModelandGetResults()
        return self.objValue, self.variableValues, self.runtime, self.gap
    
    def getUndergroundVariablesFromCSV(self):
        self.undergroundBlocksLenght = self.database['X'].to_dict()             
        self.undergroundBlocksWidth  = self.database['Y'].to_dict()             
        self.undergroundBlocksHeight = self.database['Z'].to_dict()             
        self.undergroundBlockTonnage = self.database['Ton'].to_dict()              
        self.undergroundBlockMineral  = self.database['Mineral'].to_dict()          
        self.undergroundBlockRecovery  = self.database['Recuperación'].to_dict()     
        self.undergroundCopperLaw  = self.database['%Cu'].to_dict()
        self.undergroundExtractionFixedCosts = self.database['CPlanta CA'].to_dict()
        self.undergroundVariableExtractionCosts = self.database['CMina CA'].to_dict()
        self.undergroundCP_S = self.database['CPlanta S'].to_dict()
        self.undergroundCM_S = self.database['CMINA S'].to_dict() 
    
    def getUndergroundInfo(self):
        self.undergroundBlocks = [i for i in range(len(self.undergroundBlocksLenght.values()))]

    def setUndergroundParameters(self):
        #Underground Parameters
        self.t_S   = {period : period + 1 for period in range(self.numberOfPeriods)}
        self.MU_mt = {period : 25806600.0  for period in range(self.numberOfPeriods)} #Tonleage es mina
        self.ML_mt = {period : 17204400.0  for period in range(self.numberOfPeriods)}
        self.MU_pt = {period : 17777880.0   for period in range(self.numberOfPeriods)}#Mineral es planta
        self.ML_pt = {period : 17204400.0 for period in range(self.numberOfPeriods)}
        self.qU_dt = {period : 1 for period in range(self.numberOfPeriods)}
        self.qL_dt = {period : 0 for period in range(self.numberOfPeriods)}
        self.A_d   = {period : 2 for period in range(self.numberOfPeriods)}
        self.NU_nt = {period : 59 for period in range(self.numberOfPeriods)} 
        self.NL_nt = {period : 32 for period in range(self.numberOfPeriods)}
        self.N_t   = {period : 57* (1 + period) for period in range(self.numberOfPeriods)}
        self.RL_dt = {period : 0.3 for period in range(self.numberOfPeriods)}
        self.RU_dt = {period : 0.7 for period in range(self.numberOfPeriods)}

    def setUndergroundVariables(self):
        self.drawpoint, self.G_d, self.Q_d,self.LEY_D, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw = drawpointFunction(
                        self.pos_x, self.pos_y, self.pos_z, self.colHeight, self.DP_init, self.undergroundBlocksLenghtLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits, self.undergroundBlockTonnage, self.undergroundCP_S, self.undergroundCM_S, self.undergroundBlockMineral,
                        self.undergroundCopperLaw, self.pos_x_f, self.pos_y_f,self.orientationToExtractTheDrawpoints)
    
    def setUndergroundMineLimits(self):
        self.undergroundBlocksLenghtLimits = getNumberOfBlocksInADimension(self.undergroundBlocksLenght)
        self.undergroundBlocksWidthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksWidth)
        self.undergroundBlocksHeightLimits = getNumberOfBlocksInADimension(self.undergroundBlocksHeight)

    def setModelandGetResults(self):
        self.objValue, self.variableValues, self.runtime, self.gap = self.setUndergroundModel()#v_p, theta_opt, w_opt)
    def setUndergroundModel(self):#,v_p, theta_opt, w_opt):
        print("Si funciono")
        undergroundModel = gp.Model(name = 'Modelo Integrado')
        undergroundModel.Params.TimeLimit = 3600

        #14. Naturaleza de las variables
        x_dt = undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="x")
        y_dt = undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.CONTINUOUS, name="y")
        z_dt = undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="z")

        #1. Restricción sobre la cantidad de tonelaje máxima y mínima a extraer en cada periodo.
        Ton_Up = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) <= self.MU_mt[ti] for ti in self.t_S),
                                         "Min_max")
        
        Ton_low = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) >= self.ML_mt[ti] for ti in self.t_S),
                                            "Min_min")
        #2. Restricción sobre la cantidad de material máxima y mínima a procesar en cada periodo.
        Mat_Up = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]* self.Q_d[d] for d in self.drawpoint) <= self.MU_pt[ti] for ti in self.t_S),
                                            "Mat_max")

        Mat_low = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]* self.Q_d[d] for d in self.drawpoint) >= self.ML_pt[ti] for ti in self.t_S)
                                            , "Mat_min")
        #3. Rango de leyes máximas y mínimas a procesar
        GQC_Up = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.LEY_D[d]*self.G_d[d] for d in self.drawpoint) <=
                                self.qU_dt[ti] * gp.quicksum(self.Q_d[d] * y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_Up")

        GQC_low = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.LEY_D[d]*self.G_d[d] for d in self.drawpoint) >=
                                self.qL_dt[ti] * gp.quicksum(self.Q_d[d] * y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_low")

        #4. Todos los puntos de extracci ́on deben ser iniciados en el largo de la extracción
        Drawp_init = undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint), "Drawp_init")

        #5. Los puntos de extracción deben ser activados al menos en el mismo periodo para que se inicie la extracción 
        Drawpextract_61 = undergroundModel.addConstrs((gp.quicksum(x_dt[d, tau] for tau in range(ti+1)) >= z_dt[d, ti]  
                                            for d in self.drawpoint for ti in self.t_S), "Drawpextract_61")


        #6. Existe una cantidad máxima y mínima de drawpoints a abrir en cada periodo.
        Drawpextract_64_1 = undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[ti] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_1")

        Drawpextract_64_2 = undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) >= self.NL_nt[ti] for ti 
                                                        in self.t_S)
                                                        , "Drawpextract_64_2")

        #7. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        Drawpextract_65 = undergroundModel.addConstrs((gp.quicksum(z_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")


        #8. Si iniciamos la extracción de un drawpoint esta debe durar por su duraci ́on determinada.
        ## Un drawpoint solamente puede ser extraido por un preiodo pre determinado (A_d)
        Drawpextract_62 = undergroundModel.addConstrs((gp.quicksum(z_dt[d, ti] for ti in self.t_S)  <= self.A_d[ti]  for d in self.drawpoint
                                                    for ti in self.t_S), "Drawp_62")

        ## Una vez se inicia extrayendo de un drawpoint, se continua extrayendo sin interrupción
        Drawpextract_63 = undergroundModel.addConstrs((self.A_d[ti] *(z_dt[d, ti] - z_dt[d, ti+1]) 
                                            - gp.quicksum(z_dt[d, tau] for tau in range(ti+1)) <= 0 
                                            for d in self.drawpoint for ti in range(0,max(self.t_S))), "Drawpextract_63")

        #9. Relación de variables, el porcentaje a extraer es 0 si no se extra un drawpoint.
        Drawpextract_66 = undergroundModel.addConstrs((y_dt[d, ti] <= z_dt[d, ti] for d in self.drawpoint for ti in self.t_S),
                                                    "Drawpextract_66")

        #10. Existe una tasa m ́ınima de extracci ́on para cada drawpoint a extraer.
        Drawpextract_67_1 = undergroundModel.addConstrs((self.RL_dt[ti] * z_dt[d, ti]  <=  y_dt[d, ti] for d in self.drawpoint
                                                        for ti in self.t_S), "Drawpextract_67_1")

        #11. La altura a extraer debe ser mayor a una cantidad m ́ınima.
        #rest_11 = undergroundModel.addConstrs((gp.quicksum(y_dt[d,ti] for ti in self.t_S)>= self.colHeight for d in self.drawpoint))

        #12. No podemos extraer más del 100 % de un drawpoint.
        Reserver_cnst = undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint),
                                                    "Reserver_cnst")

        #13. Si se activa un drawpoint, se extrae en ese periodo
        rest_13 = undergroundModel.addConstrs(x_dt[d,ti] <= z_dt[d, ti] for d in self.drawpoint for ti in self.t_S)

        #14. Naturaleza de variables.

        #15. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        rest_15= undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")

        

        #16. Restricción sobre el inicio de la extracci ́on de los drawpoints.
        DP_Sup = undergroundModel.addConstrs((gp.quicksum(x_dt[self.predecessor[l][0], s]*(max(self.t_S)-s+1) for s in self.t_S) <=
                                    gp.quicksum(x_dt[self.predecessor[l][1], s]*(max(self.t_S)-s+1) for s in self.t_S)  
                                    for l in range(len(self.predecessor))), "DP_Sup")

        #17. Restricci ́on sobre la extracci ́on de los drawpoints.
        restricion_z_dt = undergroundModel.addConstrs((gp.quicksum(z_dt[self.predecessor[l][0], s]*(max(self.t_S)-s+1) for s in self.t_S) <=
                                    gp.quicksum(z_dt[self.predecessor[l][1], s]*(max(self.t_S)-s+1) for s in self.t_S)  
                                    for l in range(len(self.predecessor))), "DP_Sup")



        """
        #Restricciones de partida.

        ## El número de nuevos drawpoints debe tener un limite inferior respecto al origen
        Drawpextract_64_3 = undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[0] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_3")
        restricion_partida_1 = undergroundModel.addConstr(x_dt[self.predecessor[0][0],0] == 1, "restriccion_partida 1")

        ## Lo mínimo a extraer en el primer drawpoint es de 0.3 
        restricion_partida_2 = undergroundModel.addConstr(y_dt[self.predecessor[0][0],0] >= 0.3, "restriccion_partida 2")
        
        ## Se debe extraer un mínimo de un 80% de cada drawpoint
        restricion_partida_E =undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*z_dt[d, ti] for ti in self.t_S) >= 0.8
                                                       for d in self.drawpoint), "Reserver_cnst")

        """
        
        #Función objetivo
        undergroundObjectiveFunction = gp.quicksum(y_dt[d, ti]*((((self.p_t * self.LEY_D[d] -self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        ((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)
        
        """
        #FALTA DEFINIR LOS CONJUNTOS D_v, V

        
        #Restricciones del crown pillar

        #Variable 1 si y solo si el crown pillar esta ubicado en la elevaci ́on v, 0 en otro caso.
        w_v = undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="w")

        #Restricciones del crown pillar
        pillar_2 = undergroundModel.addConstrs(gp.quicksum(x_dt[d, ti] for d in D_v)<=1- w_v[v] for v in V for ti in self.t_S)

        pillar_3 = undergroundModel.addConstrs(gp.quicksum(w_v[v])==1 for v in V)

        #Restricción extra para iterar
        if v_p < theta_opt - self.epsilon:
            optimal_cut = undergroundModel.addConstrs(theta <= v_p + gp.quicksum(mu_v*(w_v[v]-w_opt[v])for v in V))
        """
       
        undergroundModel.setObjective(undergroundObjectiveFunction, GRB.MAXIMIZE)
        undergroundModel.Params.MIPGap = 0.01
        
        undergroundModel.optimize()
        lista_variable_Integrado = (undergroundModel.getAttr(GRB.Attr.X, undergroundModel.getVars()))
        solucion = undergroundModel.objVal
        runtime = undergroundModel.Runtime
        gap_f = undergroundModel.MIPGap
        #print(y_dt)
        return solucion, lista_variable_Integrado, runtime, gap_f

In [5]:
undergroundMineDataframe = pd.read_excel("C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/Modelo_F_OG.xlsx", engine="openpyxl")

In [25]:
numberOfPeriods = 4

In [29]:
undergroundModel = UndergroundModel(undergroundMineDataframe,numberOfPeriods)

In [30]:
undergroundObjValue, undergroundVariableValues, undergroundRuntime, undergroundGap = undergroundModel.execute()

Si funciono
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 4898 rows, 2160 columns and 22844 nonzeros
Model fingerprint: 0xe935fd17
Variable types: 720 continuous, 1440 integer (1440 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+05]
  Objective range  [7e+05, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]
Presolve removed 4514 rows and 1785 columns
Presolve time: 0.04s
Presolved: 384 rows, 375 columns, 1530 nonzeros
Variable types: 296 continuous, 79 integer (79 binary)
Found heuristic solution: objective 1.909349e+09
Found heuristic solution: objective 1.919971e+09
Found heuristic solution: objective 1.923265e+09

Root relaxation: objective 1.932976e+09, 307 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      

In [41]:
undergroundModel.predecessor

[[1, 0],
 [15, 1],
 [2, 15],
 [16, 2],
 [30, 16],
 [3, 30],
 [17, 3],
 [31, 17],
 [45, 31],
 [4, 45],
 [18, 4],
 [32, 18],
 [46, 32],
 [60, 46],
 [5, 60],
 [19, 5],
 [33, 19],
 [47, 33],
 [61, 47],
 [75, 61],
 [6, 75],
 [20, 6],
 [34, 20],
 [48, 34],
 [62, 48],
 [76, 62],
 [90, 76],
 [7, 90],
 [21, 7],
 [35, 21],
 [49, 35],
 [63, 49],
 [77, 63],
 [91, 77],
 [105, 91],
 [8, 105],
 [22, 8],
 [36, 22],
 [50, 36],
 [64, 50],
 [78, 64],
 [92, 78],
 [106, 92],
 [120, 106],
 [9, 120],
 [23, 9],
 [37, 23],
 [51, 37],
 [65, 51],
 [79, 65],
 [93, 79],
 [107, 93],
 [121, 107],
 [135, 121],
 [10, 135],
 [24, 10],
 [38, 24],
 [52, 38],
 [66, 52],
 [80, 66],
 [94, 80],
 [108, 94],
 [122, 108],
 [136, 122],
 [150, 136],
 [11, 150],
 [25, 11],
 [39, 25],
 [53, 39],
 [67, 53],
 [81, 67],
 [95, 81],
 [109, 95],
 [123, 109],
 [137, 123],
 [151, 137],
 [165, 151],
 [12, 165],
 [26, 12],
 [40, 26],
 [54, 40],
 [68, 54],
 [82, 68],
 [96, 82],
 [110, 96],
 [124, 110],
 [138, 124],
 [152, 138],
 [166, 152],
 

In [ ]:
self.drawpoint, self.G_d, self.Q_d,self.LEY_D, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw 